# Projet portant sur les réseaux de neurones
Réalisé par Fréjoux Gaëtan et Niord Mathieu

## 1. Développement d'un perceptron
α sera égal à 0.1

### 1.1. Mise en place d'un perceptron simple

* *Créer la fonction `perceptron_simple(x, w, active)`*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ALPHA = 0.1  # learning rate

def perceptron_simple(x, w, active):
  seuil = w[0]
  dot = np.dot(x, w[1:])
  x = seuil + dot
  return np.sign(x) if (active == 0) else np.tanh(x)


def plot_with_class(X, weight, c, title):
    x = np.linspace(-1, 2)
    y = (weight[0] + x*weight[1]) / (-weight[2])
    plt.title(title)
    plt.scatter(X[:, 0], X[:, 1], c=c)
    plt.plot(x, y, 'r-')
    plt.grid()
    plt.show()

* *Tester votre perceptron avec l'exemple du OU logique vu en cours (en utilisant la fonction `sign(x)` comme fonction d'activation)*

In [ ]:
#TODO Test

* *Afficher dans le cadre de l'exemplo du OU logique sur la même figure les différents éléments de l'ensemble d'apprentissage et la droite séparatrice associée aux poids du neurone sur la même figure*

In [ ]:
#TODO Affichage

### 1.2. Etude de l'apprentissage

#### 1.2.1. Programmation apprentissage Widrow-Hoff

* *Créer la fonction `apprentissage_widrow(x, yd, epoch, batch_size)`. La droite séparatrice et les points d'apprentissage doivent être affichés à chaque itération (une itération correspond à la présentation de tous les individus de l'ensemble d'apprentissage), ains que l'erreur de classification*

In [ ]:
#TODO implémentation

#### 1.2.2. Test 1 simple

* *Charger les données p2_d1.txt*

In [ ]:
#TODO Chargement des données

* *Appliquer l'algorithme d'apprentissage sur les données. Afficher l'évolution de l'erreur. Vérifier que la frontière est correcte*

In [ ]:
#TODO Apprentissage

TODO commentaires

#### 1.2.3. Test 2

* *Charger les données p2_d2.txt*

In [ ]:
#TODO Chargement des données

* *Appliquer l'algorithme d'apprentissage sur les données. Afficher l'évolution de l'erreur. Vérifier que la frontière est correcte*

In [ ]:
#TODO Apprentissage

TODO commentaires

* *Comparer les résultats avec les premières données et conclure*

TODO comparaison + conclusion

### 1.3 Perceptron multicouches

#### 1.3.1. Mise en place d'un perceptron multicouche

* *Créer la fonction `multiperceptron(x, w1, w2)`*

In [ ]:
#TODO implémentation

* *Tester votre perceptron multicouches avec l'exemple ci-dessous pour un entrée x = [1 1]' :*  
<img src="exemple_1_3_1.jpg" width="600">

In [ ]:
#TODO Test

#### 1.3.2. Programmation apprentissage multicouches

* *Créer une fonction `multiperceptron_widrow(x, yd, epoch, batch_size)`*

In [ ]:
#TODO implémentation

* *Créer l'ensemble d'apprentissage*

In [ ]:
#TODO

* *Afficher cet ensemble avec la fonction affiche_classe*

In [ ]:
#TODO

* *Pensez-vous que ce problème puisse être traité par un perceptron simple ?*

In [ ]:
#TODO

* *Appliquer votre algorithme d'apprentissage*

In [ ]:
#TODO

* *Tester, à partir de votre fonction `multiperceptron`, le réseau de neurones ainsi obtenu sur l'ensemble d'apprentissage*

In [ ]:
#TODO

* *Afficher les droites séparatrices associées aux différents neurones et les points de l'ensemble d'apprentissage*

In [ ]:
#TODO

## 2. Deep et Full-connected : discrimination d'une image

### 2.1. Approche basée Descripteurs (basé modèle)

#### 2.1.1. Calcul des descripteurs

* *Importer les différents tableaux de mesure et créer un vecteur de label indiquant la classe sous forme d'un chiffre de chauqe image*

In [ ]:
#TODO

#### 2.1.2. Mise en place d'un système de discrimination basé structure Full-Connected

* *Mettre en place un système de discrimination qui pour la présentation d'une image inconnue et de son vecteur de mesures associé propose une classe*

* *Tester la procédure de la classification avec les 5 types de mesures et différentes images incoonues*

In [ ]:
#TODO

* *Analyser les résultats (matrice de confusion, taux d'erreur)*

In [ ]:
#TODO

TODO commentaires

* *Comparer avec différents hyperparamètres ????(nombre de neurones, nombre de couches, nombre d'itérations, taille du batch)????*

In [ ]:
#TODO

TODO commentaires

#### 2.1.3. Approche "Deep" (basée Data)

* *Mettre en place un système de classification qui a en données une image, calcul les descripteurs par des couches de convolution*

In [ ]:
#TODO

* *Tester des structures simples*

* *Etudier l'influence des paramètres*

* *Etudier l'évolution de la fonction de coût*

* *Comparer avec les résultats avec les méthodes basées caractéristiques*

* *Tester avec des structures plus complexes. La "Data augmentation" doit probablement être utilisée*

In [ ]:
#TODO